In [1]:
spark_spam = sc.textFile('data/spambase/spambase.data')

In [2]:
spark_spam.take(5)

[u'0,0.64,0.64,0,0.32,0,0,0,0,0,0,0.64,0,0,0,0.32,0,1.29,1.93,0,0.96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.778,0,0,3.756,61,278,1',
 u'0.21,0.28,0.5,0,0.14,0.28,0.21,0.07,0,0.94,0.21,0.79,0.65,0.21,0.14,0.14,0.07,0.28,3.47,0,1.59,0,0.43,0.43,0,0,0,0,0,0,0,0,0,0,0,0,0.07,0,0,0,0,0,0,0,0,0,0,0,0,0.132,0,0.372,0.18,0.048,5.114,101,1028,1',
 u'0.06,0,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,0.38,0.45,0.12,0,1.75,0.06,0.06,1.03,1.36,0.32,0.51,0,1.16,0.06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.06,0,0,0.12,0,0.06,0.06,0,0,0.01,0.143,0,0.276,0.184,0.01,9.821,485,2259,1',
 u'0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0,0,0.31,0,0,3.18,0,0.31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.137,0,0.137,0,0,3.537,40,191,1',
 u'0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0,0,0.31,0,0,3.18,0,0.31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.135,0,0.135,0,0,3.537,40,191,1']

In [3]:
spark_spam.count()

4601

In [5]:
import re

In [6]:
names = re.findall('^(.+):', open('data/spambase/spambase.names').read(), re.MULTILINE)

In [8]:
del names[0]
names.append('label')

In [9]:
names

['word_freq_make',
 'word_freq_address',
 'word_freq_all',
 'word_freq_3d',
 'word_freq_our',
 'word_freq_over',
 'word_freq_remove',
 'word_freq_internet',
 'word_freq_order',
 'word_freq_mail',
 'word_freq_receive',
 'word_freq_will',
 'word_freq_people',
 'word_freq_report',
 'word_freq_addresses',
 'word_freq_free',
 'word_freq_business',
 'word_freq_email',
 'word_freq_you',
 'word_freq_credit',
 'word_freq_your',
 'word_freq_font',
 'word_freq_000',
 'word_freq_money',
 'word_freq_hp',
 'word_freq_hpl',
 'word_freq_george',
 'word_freq_650',
 'word_freq_lab',
 'word_freq_labs',
 'word_freq_telnet',
 'word_freq_857',
 'word_freq_data',
 'word_freq_415',
 'word_freq_85',
 'word_freq_technology',
 'word_freq_1999',
 'word_freq_parts',
 'word_freq_pm',
 'word_freq_direct',
 'word_freq_cs',
 'word_freq_meeting',
 'word_freq_original',
 'word_freq_project',
 'word_freq_re',
 'word_freq_edu',
 'word_freq_table',
 'word_freq_conference',
 'char_freq_;',
 'char_freq_(',
 'char_freq_[',
 '

In [10]:
ord('$')

36

In [11]:
def sub_fun(match):
    return 'a' + str(ord(match.group(0)))

In [12]:
re.sub('[^a-z0-9_]', sub_fun, 'char_freq_#')

'char_freq_a35'

In [13]:
spark_names = [re.sub('[^a-z0-9_]', sub_fun, name) for name in names]

In [14]:
spark_names

['word_freq_make',
 'word_freq_address',
 'word_freq_all',
 'word_freq_3d',
 'word_freq_our',
 'word_freq_over',
 'word_freq_remove',
 'word_freq_internet',
 'word_freq_order',
 'word_freq_mail',
 'word_freq_receive',
 'word_freq_will',
 'word_freq_people',
 'word_freq_report',
 'word_freq_addresses',
 'word_freq_free',
 'word_freq_business',
 'word_freq_email',
 'word_freq_you',
 'word_freq_credit',
 'word_freq_your',
 'word_freq_font',
 'word_freq_000',
 'word_freq_money',
 'word_freq_hp',
 'word_freq_hpl',
 'word_freq_george',
 'word_freq_650',
 'word_freq_lab',
 'word_freq_labs',
 'word_freq_telnet',
 'word_freq_857',
 'word_freq_data',
 'word_freq_415',
 'word_freq_85',
 'word_freq_technology',
 'word_freq_1999',
 'word_freq_parts',
 'word_freq_pm',
 'word_freq_direct',
 'word_freq_cs',
 'word_freq_meeting',
 'word_freq_original',
 'word_freq_project',
 'word_freq_re',
 'word_freq_edu',
 'word_freq_table',
 'word_freq_conference',
 'char_freq_a59',
 'char_freq_a40',
 'char_freq_a9

In [16]:
from pyspark.sql import Row

In [18]:
def line_to_row(line):
    values = [float(i) for i in line.split(',')]
    argdict = {name: value for name, value in zip(spark_names, values)}
    return Row(**argdict)

In [19]:
df = spark.createDataFrame(spark_spam.map(line_to_row))

In [21]:
df.select(spark_names[:5]).show(5)

+--------------+-----------------+-------------+------------+-------------+
|word_freq_make|word_freq_address|word_freq_all|word_freq_3d|word_freq_our|
+--------------+-----------------+-------------+------------+-------------+
|           0.0|             0.64|         0.64|         0.0|         0.32|
|          0.21|             0.28|          0.5|         0.0|         0.14|
|          0.06|              0.0|         0.71|         0.0|         1.23|
|           0.0|              0.0|          0.0|         0.0|         0.63|
|           0.0|              0.0|          0.0|         0.0|         0.63|
+--------------+-----------------+-------------+------------+-------------+
only showing top 5 rows



In [23]:
from pyspark.ml.feature import RFormula

In [25]:
formula = 'label ~ word_freq_address + word_freq_3d + word_freq_direct'

In [27]:
selector = RFormula(formula=formula)

In [28]:
X_formula = selector.fit(df).transform(df)

In [31]:
X_formula.select('features', 'label').show(5)

+--------------+-----+
|      features|label|
+--------------+-----+
|[0.64,0.0,0.0]|  1.0|
|[0.28,0.0,0.0]|  1.0|
|[0.0,0.0,0.06]|  1.0|
|     (3,[],[])|  1.0|
|     (3,[],[])|  1.0|
+--------------+-----+
only showing top 5 rows



In [32]:
X_formula.schema['features']

StructField(features,VectorUDT,true)

In [33]:
from pyspark.ml.feature import VectorAssembler

In [34]:
input_features = spark_names[:10]

In [35]:
input_features

['word_freq_make',
 'word_freq_address',
 'word_freq_all',
 'word_freq_3d',
 'word_freq_our',
 'word_freq_over',
 'word_freq_remove',
 'word_freq_internet',
 'word_freq_order',
 'word_freq_mail']

In [36]:
X_feat = (VectorAssembler(inputCols=input_features, outputCol='my_features')
         .transform(df).select('my_features'))

In [38]:
X_feat.show(5)

+--------------------+
|         my_features|
+--------------------+
|(10,[1,2,4],[0.64...|
|[0.21,0.28,0.5,0....|
|[0.06,0.0,0.71,0....|
|(10,[4,6,7,8,9],[...|
|(10,[4,6,7,8,9],[...|
+--------------------+
only showing top 5 rows



In [43]:
X_feat.schema['my_features'].metadata

{u'ml_attr': {u'attrs': {u'numeric': [{u'idx': 0, u'name': u'word_freq_make'},
    {u'idx': 1, u'name': u'word_freq_address'},
    {u'idx': 2, u'name': u'word_freq_all'},
    {u'idx': 3, u'name': u'word_freq_3d'},
    {u'idx': 4, u'name': u'word_freq_our'},
    {u'idx': 5, u'name': u'word_freq_over'},
    {u'idx': 6, u'name': u'word_freq_remove'},
    {u'idx': 7, u'name': u'word_freq_internet'},
    {u'idx': 8, u'name': u'word_freq_order'},
    {u'idx': 9, u'name': u'word_freq_mail'}]},
  u'num_attrs': 10}}

In [46]:
spark_names[-1]

'label'

In [49]:
all_feat = spark_names
del all_feat[-1]
X_all = (VectorAssembler(inputCols=all_feat, outputCol='features')
         .transform(df).select('features', 'label'))

In [50]:
X_all.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(55,[1,2,4,11,15,...|  1.0|
|(55,[0,1,2,4,5,6,...|  1.0|
|(55,[0,2,4,5,6,7,...|  1.0|
|(55,[4,6,7,8,9,10...|  1.0|
|(55,[4,6,7,8,9,10...|  1.0|
+--------------------+-----+
only showing top 5 rows



In [44]:
from pyspark.ml.feature import ChiSqSelector

In [51]:
cs_selector = ChiSqSelector(numTopFeatures=10, featuresCol='features',
                            outputCol='selected_features')

In [53]:
X_chis = cs_selector.fit(X_all).transform(X_all)

In [54]:
X_chis.show(5)

+--------------------+-----+--------------------+
|            features|label|   selected_features|
+--------------------+-----+--------------------+
|(55,[1,2,4,11,15,...|  1.0|(10,[1,2,3],[0.64...|
|(55,[0,1,2,4,5,6,...|  1.0|(10,[0,1,2,3,4,5,...|
|(55,[0,2,4,5,6,7,...|  1.0|(10,[0,2,3,4,5,6,...|
|(55,[4,6,7,8,9,10...|  1.0|(10,[3,5,6,7,8,9]...|
|(55,[4,6,7,8,9,10...|  1.0|(10,[3,5,6,7,8,9]...|
+--------------------+-----+--------------------+
only showing top 5 rows



In [55]:
X_chis.schema['selected_features'].metadata

{u'ml_attr': {u'attrs': {u'numeric': [{u'idx': 0, u'name': u'word_freq_make'},
    {u'idx': 1, u'name': u'word_freq_address'},
    {u'idx': 2, u'name': u'word_freq_all'},
    {u'idx': 3, u'name': u'word_freq_our'},
    {u'idx': 4, u'name': u'word_freq_over'},
    {u'idx': 5, u'name': u'word_freq_remove'},
    {u'idx': 6, u'name': u'word_freq_internet'},
    {u'idx': 7, u'name': u'word_freq_order'},
    {u'idx': 8, u'name': u'word_freq_mail'},
    {u'idx': 9, u'name': u'word_freq_receive'}]},
  u'num_attrs': 10}}

In [56]:
from pyspark.ml.classification import DecisionTreeClassifier

In [57]:
from pyspark.ml import Pipeline

In [74]:
assembler = VectorAssembler(inputCols=all_feat, outputCol='assembled_features')
selector = ChiSqSelector(numTopFeatures=30, featuresCol='assembled_features',
                            outputCol='final_features')
tree_model = DecisionTreeClassifier(maxDepth= 5, featuresCol='final_features')

In [76]:
pipe = Pipeline(stages=[assembler, selector, tree_model])

In [77]:
train, test = df.randomSplit([0.8, 0.2])

In [78]:
model = pipe.fit(train)

In [62]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [79]:
BinaryClassificationEvaluator().evaluate(model.transform(test))

0.8171707624448645

In [80]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [81]:
param_grid = (ParamGridBuilder()
             .addGrid(tree_model.maxDepth, [2,4,6])
             .addGrid(selector.numTopFeatures, [20, 40])
             .build())

In [82]:
cv = CrossValidator(estimator=pipe, estimatorParamMaps=param_grid,
                    evaluator=BinaryClassificationEvaluator(), numFolds=3)

In [83]:
cv_model = cv.fit(df)

In [84]:
cv_model.avgMetrics

[0.5394303275973474,
 0.2495034073847745,
 0.33496063484138827,
 0.7129752343464821,
 0.7550001582260496,
 0.7415963504902513]

In [86]:
import numpy as np

In [88]:
cv_model.getEstimatorParamMaps()[np.argmax(cv_model.avgMetrics)]

{Param(parent=u'ChiSqSelector_4884bab9c445e4846aa9', name='numTopFeatures', doc='Number of features that selector will select, ordered by ascending p-value. If the number of features is < numTopFeatures, then this will select all features.'): 40,
 Param(parent=u'DecisionTreeClassifier_42bf8ead4d84f787ecf9', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 4}

# NLP

In [90]:
import wikipedia

In [91]:
obama_page = wikipedia.page('Barack Obama')

In [98]:
match = re.search('president', obama_page.content)

In [100]:
match.group(0)

u'president'

In [103]:
re.findall('[Pp]resident', obama_page.content)

[u'President',
 u'president',
 u'president',
 u'president',
 u'president',
 u'President',
 u'President',
 u'President',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'President',
 u'president',
 u'President',
 u'President',
 u'President',
 u'president',
 u'President',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'President',
 u'president',
 u'president',
 u'president',
 u'president',
 u'President',
 u'President',
 u'president',
 u'president',
 u'President',
 u'President',
 u'President',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'President',
 u'President',
 u'President',
 u'President',
 u'president',
 u'President',
 u'President',
 u'President',
 u'president',
 u'President',
 u'president',
 u'President',
 u'President',
 u'President',
 u'President',
 u'President',
 u'President',
 u'president',
 u'President',
 u'President',
 u'President',
 u'preside

In [104]:
re.findall('[Pp]resident ([A-Za-z]+)', obama_page.content)

[u'of',
 u'of',
 u'to',
 u'of',
 u'have',
 u'of',
 u'of',
 u'and',
 u'Bush',
 u'Bush',
 u'of',
 u'of',
 u'he',
 u'Bill',
 u'and',
 u'in',
 u'since',
 u'Obama',
 u'took',
 u'George',
 u'Ronald',
 u'Obama',
 u'to',
 u'to',
 u'mentioned',
 u'Valerie',
 u'Joe',
 u'of',
 u'Obama',
 u'Obama',
 u'Obama',
 u'Joe',
 u'Biden',
 u'to',
 u'Dmitry',
 u'Obama',
 u'Assad',
 u'Obama',
 u'Ra',
 u'Barack',
 u'to',
 u'to',
 u'ever',
 u'to',
 u'Obama',
 u'of',
 u'to',
 u'Barack',
 u'Obama',
 u'Franklin',
 u'Bill',
 u'Obama',
 u'George',
 u'Obama',
 u'Jimmy',
 u'Ronald']

In [106]:
words = [i.lower() for i in obama_page.content.split()]

In [113]:
words[:50]

[u'barack',
 u'hussein',
 u'obama',
 u'ii',
 u'(us',
 u'/b\u0259\u02c8r\u0251\u02d0k',
 u'hu\u02d0\u02c8se\u026an',
 u'o\u028a\u02c8b\u0251\u02d0m\u0259/',
 u'b\u0259-rahk',
 u'hoo-sayn',
 u'oh-bah-m\u0259;',
 u'born',
 u'august',
 u'4,',
 u'1961)',
 u'is',
 u'an',
 u'american',
 u'politician',
 u'who',
 u'served',
 u'as',
 u'the',
 u'44th',
 u'president',
 u'of',
 u'the',
 u'united',
 u'states',
 u'from',
 u'2009',
 u'to',
 u'2017.',
 u'he',
 u'is',
 u'the',
 u'first',
 u'african',
 u'american',
 u'to',
 u'have',
 u'served',
 u'as',
 u'president,',
 u'as',
 u'well',
 u'as',
 u'the',
 u'first',
 u'born']

In [108]:
len(set(words)) / float(len(words))

0.3005591054313099

In [110]:
import nltk.data

In [111]:
from nltk import word_tokenize

In [112]:
word_tokenize(obama_page.content)

[u'Barack',
 u'Hussein',
 u'Obama',
 u'II',
 u'(',
 u'US',
 u'/b\u0259\u02c8r\u0251\u02d0k',
 u'hu\u02d0\u02c8se\u026an',
 u'o\u028a\u02c8b\u0251\u02d0m\u0259/',
 u'b\u0259-RAHK',
 u'hoo-SAYN',
 u'oh-BAH-m\u0259',
 u';',
 u'born',
 u'August',
 u'4',
 u',',
 u'1961',
 u')',
 u'is',
 u'an',
 u'American',
 u'politician',
 u'who',
 u'served',
 u'as',
 u'the',
 u'44th',
 u'President',
 u'of',
 u'the',
 u'United',
 u'States',
 u'from',
 u'2009',
 u'to',
 u'2017',
 u'.',
 u'He',
 u'is',
 u'the',
 u'first',
 u'African',
 u'American',
 u'to',
 u'have',
 u'served',
 u'as',
 u'president',
 u',',
 u'as',
 u'well',
 u'as',
 u'the',
 u'first',
 u'born',
 u'outside',
 u'the',
 u'contiguous',
 u'United',
 u'States',
 u'.',
 u'He',
 u'previously',
 u'served',
 u'in',
 u'the',
 u'U.S.',
 u'Senate',
 u'representing',
 u'Illinois',
 u'from',
 u'2005',
 u'to',
 u'2008',
 u',',
 u'and',
 u'in',
 u'the',
 u'Illinois',
 u'State',
 u'Senate',
 u'from',
 u'1997',
 u'to',
 u'2004',
 u'.',
 u'Obama',
 u'was',
 u'

In [114]:
sent_tok = nltk.data.load('tokenizers/punkt/english.pickle')

In [117]:
sent_tok.tokenize(obama_page.content)

[u'Barack Hussein Obama II (US /b\u0259\u02c8r\u0251\u02d0k hu\u02d0\u02c8se\u026an o\u028a\u02c8b\u0251\u02d0m\u0259/ b\u0259-RAHK hoo-SAYN oh-BAH-m\u0259; born August 4, 1961) is an American politician who served as the 44th President of the United States from 2009 to 2017.',
 u'He is the first African American to have served as president, as well as the first born outside the contiguous United States.',
 u'He previously served in the U.S. Senate representing Illinois from 2005 to 2008, and in the Illinois State Senate from 1997 to 2004.',
 u'Obama was born in Honolulu, Hawaii, two years after the territory was admitted to the Union as the 50th state.',
 u'He grew up mostly in Hawaii, but also spent one year of his childhood in Washington State and four years in Indonesia.',
 u'After graduating from Columbia University in 1983, he worked as a community organizer in Chicago.',
 u'In 1988 Obama enrolled in Harvard Law School, where he was the first black president of the Harvard Law Re

In [119]:
from nltk.stem.porter import PorterStemmer

In [120]:
stemmer = PorterStemmer()

In [121]:
stemmer.stem('carefully')

u'care'

In [122]:
stemmer.stem('scarred')

u'scar'

In [127]:
stemmer.stem('incapable')

u'incap'

In [124]:
float(len(set([stemmer.stem(word) for word in words]))) / len(words)

0.2594249201277955

In [128]:
from nltk.stem.wordnet import WordNetLemmatizer

In [129]:
lemmatizer = WordNetLemmatizer()

In [130]:
lemmatizer.lemmatize('cats')

u'cat'

In [133]:
lemmatizer.lemmatize('are', pos='v')

u'be'

In [134]:
lemmatizer.lemmatize('were', pos='v')

u'be'

In [135]:
from nltk.probability import ConditionalFreqDist

In [136]:
from nltk import bigrams

In [138]:
list(bigrams('after all is said and done'.split()))

[('after', 'all'),
 ('all', 'is'),
 ('is', 'said'),
 ('said', 'and'),
 ('and', 'done')]

In [139]:
cfd = ConditionalFreqDist(bigrams([word.lower() for word in word_tokenize(obama_page.content)]))

In [140]:
cfd['president']

FreqDist({u"'s": 2,
          u',': 3,
          u'.': 6,
          u'and': 2,
          u'assad': 1,
          u'barack': 2,
          u'biden': 1,
          u'bill': 2,
          u'bush': 2,
          u'dmitry': 1,
          u'ever': 1,
          u'franklin': 1,
          u'george': 2,
          u'have': 1,
          u'he': 1,
          u'in': 1,
          u'jimmy': 1,
          u'joe': 2,
          u'mentioned': 1,
          u'obama': 11,
          u'of': 9,
          u'ra\xfal': 1,
          u'ronald': 2,
          u'since': 1,
          u'to': 8,
          u'took': 1,
          u'valerie': 1})

In [141]:
cfd['senate'].max()

u'in'

In [142]:
def gen_text(word, bigram_cfd):
    while True:
        yield word
        word = cfd[word].max()

In [143]:
from itertools import islice

In [145]:
list(islice(gen_text('president', cfd), 20))

['president',
 u'obama',
 u"'s",
 u'health',
 u'care',
 u'act',
 u'of',
 u'the',
 u'first',
 u'time',
 u'in',
 u'the',
 u'first',
 u'time',
 u'in',
 u'the',
 u'first',
 u'time',
 u'in',
 u'the']

In [147]:
from nltk.probability import ConditionalProbDist, MLEProbDist

In [148]:
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [150]:
p = cpd['president']

In [151]:
p.prob('obama')

0.16417910447761194

In [154]:
cfd['president']['obama'] / float([word.lower() 
                                   for word in 
                                   word_tokenize(obama_page.content)].count('president'))

0.16417910447761194

In [156]:
p.prob('biden')

0.014925373134328358

In [161]:
def prob_sent(sent, cpd):
    words = [word.lower() for word in word_tokenize(sent)]
    p = 1
    for w1, w2 in bigrams(words):
        p *= cpd[w1].prob(w2)
    return p

In [162]:
prob_sent('health care act', cpd)

0.1818181818181818

In [163]:
def log_prob_sent(sent, cpd):
    words = [word.lower() for word in word_tokenize(sent)]
    return sum(cpd[w1].logprob(w2) for w1, w2 in bigrams(words))

In [165]:
2**log_prob_sent('health care act', cpd)

0.18181818181818182

In [168]:
import os

In [169]:
positive = [open('data/review_polarity/txt_sentoken/pos/' + text_file).read().split()
           for text_file in os.listdir('data/review_polarity/txt_sentoken/pos/')]

In [172]:
neagive = [open('data/review_polarity/txt_sentoken/neg/' + text_file).read().split()
           for text_file in os.listdir('data/review_polarity/txt_sentoken/neg/')]

In [174]:
from nltk.probability import FreqDist
from itertools import chain, ifilter

In [175]:
from nltk.corpus import stopwords

In [176]:
en_stop = set(stopwords.words('english'))

In [177]:
en_stop

{u'a',
 u'about',
 u'above',
 u'after',
 u'again',
 u'against',
 u'ain',
 u'all',
 u'am',
 u'an',
 u'and',
 u'any',
 u'are',
 u'aren',
 u'as',
 u'at',
 u'be',
 u'because',
 u'been',
 u'before',
 u'being',
 u'below',
 u'between',
 u'both',
 u'but',
 u'by',
 u'can',
 u'couldn',
 u'd',
 u'did',
 u'didn',
 u'do',
 u'does',
 u'doesn',
 u'doing',
 u'don',
 u'down',
 u'during',
 u'each',
 u'few',
 u'for',
 u'from',
 u'further',
 u'had',
 u'hadn',
 u'has',
 u'hasn',
 u'have',
 u'haven',
 u'having',
 u'he',
 u'her',
 u'here',
 u'hers',
 u'herself',
 u'him',
 u'himself',
 u'his',
 u'how',
 u'i',
 u'if',
 u'in',
 u'into',
 u'is',
 u'isn',
 u'it',
 u'its',
 u'itself',
 u'just',
 u'll',
 u'm',
 u'ma',
 u'me',
 u'mightn',
 u'more',
 u'most',
 u'mustn',
 u'my',
 u'myself',
 u'needn',
 u'no',
 u'nor',
 u'not',
 u'now',
 u'o',
 u'of',
 u'off',
 u'on',
 u'once',
 u'only',
 u'or',
 u'other',
 u'our',
 u'ours',
 u'ourselves',
 u'out',
 u'over',
 u'own',
 u're',
 u's',
 u'same',
 u'shan',
 u'she',
 u'shoul

In [178]:
fd = FreqDist(ifilter(lambda x: x not in en_stop, chain(chain(*positive), chain(*neagive))))

In [179]:
list(chain('abc', 'def'))

['a', 'b', 'c', 'd', 'e', 'f']

In [180]:
words = [w for w, c in fd.most_common(50)]

In [181]:
words

[',',
 '.',
 '"',
 ')',
 '(',
 'film',
 'one',
 'movie',
 '?',
 "it's",
 'like',
 ':',
 'even',
 'good',
 'time',
 'story',
 'would',
 'much',
 'also',
 'get',
 'character',
 ';',
 'two',
 'characters',
 'first',
 'see',
 '!',
 'way',
 'well',
 'make',
 'really',
 'films',
 'little',
 'life',
 'plot',
 'people',
 '-',
 'could',
 'scene',
 'bad',
 'never',
 'best',
 'new',
 'many',
 "doesn't",
 'scenes',
 'man',
 'know',
 "don't",
 'movies']

In [182]:
import pandas as pd
from sklearn.naive_bayes import BernoulliNB

In [185]:
data_pos = pd.DataFrame({word: [word in text for text in positive] for word in words})
data_neg = pd.DataFrame({word: [word in text for text in neagive] for word in words})

In [186]:
data_pos['label'] = 1
data_neg['label'] = 0
data = data_pos.append(data_neg)

In [187]:
from sklearn.model_selection import cross_val_score

In [190]:
data

,!,"""",(,),",",-,.,:,;,?,...,scene,scenes,see,story,time,two,way,well,would,label
0,False,True,True,True,True,False,True,True,True,False,...,False,False,False,False,False,False,True,True,False,1
1,False,False,True,True,True,False,True,True,False,True,...,True,False,False,True,True,False,True,False,True,1
2,True,True,True,True,True,False,True,False,False,True,...,False,False,False,True,True,False,False,False,False,1
3,True,True,True,True,True,False,True,True,False,True,...,True,True,True,False,True,False,False,True,True,1
4,False,True,True,True,True,False,True,False,False,True,...,False,True,True,False,False,False,True,True,False,1
5,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,1
6,False,True,True,True,True,False,True,True,True,True,...,True,True,True,False,True,False,True,True,True,1
7,False,True,True,True,True,False,True,False,False,False,...,True,False,False,False,False,True,False,True,False,1
8,False,True,True,True,True,False,True,True,False,True,...,True,False,True,True,False,False,True,True,True,1
9,True,True,True,True,True,True,True,True,True,False,...,False,True,True,False,True,False,False,True,False,1


In [189]:
cross_val_score(BernoulliNB(), data[words], data['label'], cv=10)

array([ 0.64 ,  0.665,  0.675,  0.715,  0.67 ,  0.73 ,  0.645,  0.665,
        0.68 ,  0.63 ])